# Project Selection
## Topic Selection
- **Overal idea**: Texture classification using contact microphone data from robotic hand
- **Analysis approaches**:
  - 1D Convolutional Neural Network (CNN) on time-series data
  - 2D CNN on spectrogram
  - Hidden Markov Model (HMM) on time-series
  - HMM on spectrogram
- **Problem statement**: Can CNNs outperform HMM models for texture classification tasks on acoustic contact-sensing data gathered from a robotic hand
- **Relevant resources**:
  - [AU Dataset for Visuo-Haptic Object Recognition for Robots](https://arxiv.org/pdf/2112.13761.pdf)
  - [Publicly available datatset](https://figshare.com/articles/dataset/AU_Dataset_for_Visuo-Haptic_Object_Recognition_for_Robots/14222486)
  - [Another publicly available dataset](https://arxiv.org/pdf/2307.00937.pdf)
- **Precedent literature**:
  - [Journal: identifying pill type based on acoustic data gathered via shaking motion](https://tams.informatik.uni-hamburg.de/people/jonetzko/publications/jonetzko2020multimodal.pdf)
  - [Object classification with transmission acoustic sensing (Prof. Culbertson)](https://arxiv.org/pdf/2308.01600.pdf)
  - [Fabric classification based on acoustic data gathered via sliding motion (dataset avilable)](https://www.frontiersin.org/articles/10.3389/fnbot.2022.808222/full) 

In [1]:
import torch
x = torch.rand(5, 3)
print(x)

ModuleNotFoundError: No module named 'torch'